In [9]:
# basic libraries
import itertools
import re
import string

# data processing 
import collections
import numpy as np
import pandas as pd

# natural language pre-processing 
from textblob import TextBlob
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer

# ML pipelines
from transformers import pipeline

# sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# data visualization
import seaborn as sb
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macarthur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/macarthur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# read csv
df = pd.read_csv('../csv/data_clean.csv')
df_copy = df.copy()

In [10]:
classifier = pipeline("text-classification",model='bhadresh-savani/albert-base-v2-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

Downloading: 100%|██████████| 1.11k/1.11k [00:00<00:00, 337kB/s]
Downloading: 100%|██████████| 46.8M/46.8M [00:06<00:00, 6.80MB/s]
Downloading: 100%|██████████| 1.31M/1.31M [00:00<00:00, 1.64MB/s]
Downloading: 100%|██████████| 428/428 [00:00<00:00, 210kB/s]


[[{'label': 'sadness', 'score': 0.010403607971966267}, {'label': 'joy', 'score': 0.8902178406715393}, {'label': 'love', 'score': 0.0425327830016613}, {'label': 'anger', 'score': 0.04129811003804207}, {'label': 'fear', 'score': 0.011772057972848415}, {'label': 'surprise', 'score': 0.003775620134547353}]]
